# Getting started

The purpose of this notebook is to show very quickly the main features of scikit-decide. The next notebooks are going more into details about specific problems.

<div class="toc"><ul class="toc-item"><li><span><a href="#Run-a-Gym-environment" data-toc-modified-id="Run-a-Gym-environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Run a Gym environment</a></span></li><li><span><a href="#Solve-a-Gym-environment-with-Reinforcement-Learning" data-toc-modified-id="Solve-a-Gym-environment-with-Reinforcement-Learning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Solve a Gym environment with Reinforcement Learning</a></span></li><li><span><a href="#Solve-a-Gym-environment-with-Cartesian-Genetic-Programming" data-toc-modified-id="Solve-a-Gym-environment-with-Cartesian-Genetic-Programming-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Solve a Gym environment with Cartesian Genetic Programming</a></span></li></ul></div>

##  Run a Gym environment

Import modules.

In [ ]:
import gym

from skdecide.hub.domain.gym import GymDomain
from skdecide.utils import rollout

Select a [Gym environment](https://gym.openai.com/envs) and run 5 episodes.

In [ ]:
ENV_NAME = "CartPole-v1"  # or any other installed environment ('MsPacman-v4'...)

gym_domain = GymDomain(gym.make(ENV_NAME))
rollout(
    gym_domain, num_episodes=5, max_steps=1000, max_framerate=30, outcome_formatter=None
)
gym_domain.close()  # optional but recommended to avoid Gym errors at the end

##  Solve a Gym environment with Reinforcement Learning

Import modules.

In [ ]:
import gym
from stable_baselines3 import PPO

from skdecide.hub.domain.gym import GymDomain
from skdecide.hub.solver.stable_baselines import StableBaseline
from skdecide.utils import rollout

Select a [Gym environment](https://gym.openai.com/envs) and solve it with a [Stable Baselines](https://stable-baselines3.readthedocs.io/en/master/index.html) solver wrapped in scikit-decide.
The solution is then saved (for later reuse) and assessed in rollout.

In [ ]:
ENV_NAME = "CartPole-v1"

domain_factory = lambda: GymDomain(gym.make(ENV_NAME))
domain = domain_factory()
if StableBaseline.check_domain(domain):
    solver_factory = lambda: StableBaseline(
        PPO, "MlpPolicy", learn_config={"total_timesteps": 30000}, verbose=1
    )
    with solver_factory() as solver:
        GymDomain.solve_with(solver, domain_factory)
        solver.save("TEMP_Baselines")
        rollout(
            domain,
            solver,
            num_episodes=1,
            max_steps=1000,
            max_framerate=30,
            outcome_formatter=None,
        )

Restore saved solution and re-run rollout.

In [ ]:
with solver_factory() as solver:
    GymDomain.solve_with(solver, domain_factory, load_path="TEMP_Baselines")
    rollout(
        domain,
        solver,
        num_episodes=1,
        max_steps=1000,
        max_framerate=30,
        outcome_formatter=None,
    )

##  Solve a Gym environment with Cartesian Genetic Programming

Import modules.

In [ ]:
import gym

from skdecide.hub.domain.gym import GymDomain
from skdecide.hub.solver.cgp import CGP  # Cartesian Genetic Programming
from skdecide.utils import rollout

Select a [Gym environment](https://gym.openai.com/envs) and solve it with Cartesian Genetic Programming in scikit-decide.
The solution is then assessed in rollout.

In [ ]:
ENV_NAME = "MountainCarContinuous-v0"

domain_factory = lambda: GymDomain(gym.make(ENV_NAME))
domain = domain_factory()
if CGP.check_domain(domain):
    solver_factory = lambda: CGP("TEMP_CGP", n_it=25)
    with solver_factory() as solver:
        GymDomain.solve_with(solver, domain_factory)
        rollout(
            domain,
            solver,
            num_episodes=5,
            max_steps=1000,
            max_framerate=30,
            outcome_formatter=None,
        )